In [19]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr, spearmanr, normaltest
from sklearn.cluster import KMeans
import numpy as np
from pathlib import Path

In [2]:
os.chdir("/media/feike/14Tdisk1/ZHY") 
print(os.getcwd())

/media/feike/14Tdisk1/ZHY


In [4]:
base_path = './output/model_feature_selection/T_test_2/'

In [5]:
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.xlsx') or file.endswith('.xls'):
            file_path = os.path.join(root, file)
            df = pd.read_excel(file_path)
            if "病例完成治疗的周期数" in df.columns:
                df.rename(columns={"病例完成治疗的周期数": "treatment_cycle"}, inplace=True)
                df.to_excel(file_path, index=False)
                print(f"Updated file: {file_path}")
            else:
                print(f"Column not found in {file_path}")

Column not found in ./output/model_feature_selection/T_test_2/model2_ml/ml_features_train_t_test_p0.05.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model2_ml/ml_features_validation.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model3_dl/dl_features_train_t_test_p0.05.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model3_dl/dl_features_validation.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model4_cl+ml/cl_ml_features_train_t_test_p0.05.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model4_cl+ml/cl_ml_features_validation.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model5_cl+dl/cl_dl_features_train_t_test_p0.05.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model5_cl+dl/cl_dl_features_validation.xlsx
Column not found in ./output/model_feature_selection/T_test_2/model6_cl+ml+dl/cl_ml_dl_features_train_t_test_p0.05.xlsx
Column not found in ./ou

In [7]:
def process_selected_file(file_path):
    df = pd.read_excel(file_path) if file_path.endswith('.xlsx') else pd.read_csv(file_path) 

    X = df.iloc[:, 2:].values 
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    is_normal = True 
    significant_columns = df.columns[2:]
    for col in significant_columns:
        if not normaltest(df[col])[1] >= 0.05:
            is_normal = False
            break

    corr_func = pearsonr if is_normal else spearmanr  
    corr_values = []
    for col1 in significant_columns:
        corr_row = []
        for col2 in significant_columns:
            corr_val, _ = corr_func(df[col1], df[col2])
            corr_row.append(corr_val)
        corr_values.append(corr_row)
    corr_df = pd.DataFrame(corr_values, columns=significant_columns, index=significant_columns)

    kmeans = KMeans(n_clusters=3) 
    y = kmeans.fit_predict(corr_df.values.T)
    new_order = y.argsort()
    corr_df = corr_df.iloc[new_order, :].iloc[:, new_order]

    correlated_features = set() 
    for i in range(len(corr_df.columns)):
        for j in range(i):
            if abs(corr_df.iloc[i, j]) > 0.9:
                colname = corr_df.columns[i]
                correlated_features.add(colname)

    selected_cols = [col for col in significant_columns if col not in correlated_features]
    selected_data = df[['Group', 'imageName'] + selected_cols]
    
    output_filename = f"{os.path.basename(file_path)}" # save
    output_filepath = os.path.join("./output/model_feature_selection/T_test_2/relevance_selection", output_filename)
    selected_data.to_excel(output_filepath, index=False)

    selected_data_numeric = selected_data.select_dtypes(include=[np.number]) 
    plt.figure(figsize=(30, 30)) 
    sns.heatmap(selected_data_numeric.corr(), annot=False, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f'Correlation Heatmap of Selected Features ({file_path})')
    #plt.yticks(rotation=45)  
    #plt.xticks(rotation=45)  
    plt.savefig(os.path.join("./output/model_feature_selection/T_test_2/relevance_selection", f"heatmap_{os.path.basename(file_path).split('.')[0]}.png"))
    plt.savefig(os.path.join("./output/model_feature_selection/T_test_2/relevance_selection", f"heatmap_{os.path.basename(file_path).split('.')[0]}.pdf"), dpi=500)
    plt.close()  

    print(f"Finished processing: {file_path}")

In [9]:
data_dir = "./output/model_feature_selection/T_test_2" 
if not os.path.exists(data_dir):
    print("Error: directory does not exist in the current path.")
else:
    for root, _, files in os.walk(data_dir): 
        for file in files:
            if ('features_train' in file) and (file.endswith('.xlsx') or file.endswith('.csv')):
                file_path = os.path.join(root, file)
                process_selected_file(file_path)

Finished processing: ./output/model_feature_selection/T_test_2/model2_ml/ml_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/model3_dl/dl_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/model4_cl+ml/cl_ml_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/model5_cl+dl/cl_dl_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/model6_cl+ml+dl/cl_ml_dl_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/relevance_selection/cl_dl_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/relevance_selection/cl_ml_dl_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2/relevance_selection/cl_ml_features_train_t_test_p0.05.xlsx
Finished processing: ./output/model_feature_selection/T_test_2

In [17]:
directory_path = './output/model_feature_selection/T_test_2/relevance_selection'

In [20]:
for file_name in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, file_name)):
        old_file_path = Path(directory_path) / file_name
        name, ext = os.path.splitext(file_name)
        new_file_name = f"{name}_FS{ext}"
        new_file_path = Path(directory_path) / new_file_name
        os.rename(old_file_path, new_file_path)

文件重命名完成。
